### Dataset

This script gives insights about the dataset used for the web application. You can find the original dataset here:
https://zenodo.org/records/7858848

Since the audio files can not be published due to privacy reasons, we recommend to use the dataset.json shipped with this repository to try out the application. It is ready to use without the need of this script. If you want to use your own dataset, you can use this script as a guideline on how to sturcutre the dataset and on how to extract the acoustic features used in the aplication


##### This is what a dataset should look like:

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# load dataset
path = '../dataset.json'
dataset = pd.read_json(path, dtype={'file_name': 'str'})
dataset.head(2)

- if some of these Items are not in your dataset, you might want to create them and set them to nan, 0 or "undefined"

- if items are missing, some functionalities in the webapp will not be working 

- all soundscape items are scaled from 0-4. The sliders in the web app are also sclaed from 0-4 (except the acoustic features)
- --> The ranges of the acoustic features are the min and max value of each feature in the dataset. You have to set the slider ranges of the acoustic features manually in the webapp according to your dataset
- --> set min max values in frontend/src/components/SliderRanges/sliderSoundscapeComponent.js


In [ ]:
# get min max ranges of acoustic features
# --> set min max values in frontend/src/components/SliderRanges/sliderSoundscapeComponent.js

col = ['LAeq_default', 'N5_default', 'FavgArith_default', 'RAavgArith', 'SavgArith_default', 'R_default', 'T_default']

min_max_values = dataset[col].agg({'min', 'max'})
min_max_values

#### Create Dataset based on publication (zenodo repository, see link above)

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# load dataset
path = '../dataset/02 Dataset.csv' # the original dataset from the zenodo repository
df = pd.read_csv(path, sep=';')

# create unique audio id
dataset = df.copy()
audio_id = dataset['ID'].astype(str) + '_' + dataset['Trigger_counter'].astype(str)
dataset.insert(2, 'file_name', audio_id)
dataset.head()

In [ ]:
# keep only specific columns
to_keep = ['file_name', 'Soundscape_eventfulness', 'Soundscape_pleasantness', 'BGpleasant', 'BGchaotic', 'BGvibrant', 'BGuneventful', 'BGcalm', 'BGannoying', 
           'BGeventful', 'BGmonotonous', 'SC_Nature', 'SC_Human', 'SC_Household', 'SC_Installation', 'SC_Signals', 'SC_Traffic', 'SC_Speech', 
           'SC_Music', 'FGsource', 'Activity',  'Location8']

dataset = dataset[to_keep]
dataset.head()

In [ ]:
# rename columns
to_rename = {'Soundscape_eventfulness':'ISO_Eventfulness', 'Soundscape_pleasantness': 'ISO_Pleasantness', 
             'BGpleasant':'pleasant', 'BGchaotic':'chaotic', 'BGvibrant':'vibrant', 'BGuneventful':'uneventful', 
             'BGcalm':'calm', 'BGannoying':'annoying', 'BGeventful':'eventful', 'BGmonotonous':'monotonous'}

dataset.rename(columns=to_rename, inplace=True)
dataset.head()

In [ ]:
# add duration in seconds and suffix:
dataset.insert(1, 'duration_s', 15)
dataset.insert(2, 'suffix', '.wav')
dataset.head()

In [ ]:
# create new value ranges of soundscape items
def range_zero_to_four(x):
    return (x / (4 + np.sqrt(32)) + 1) * 2

def sc_range(x):
    x = round(x * 0.4, 1)

    return x.astype(float)

dataset['ISO_Eventfulness'] = dataset['ISO_Eventfulness'].apply(range_zero_to_four)
dataset['ISO_Pleasantness'] = dataset['ISO_Pleasantness'].apply(range_zero_to_four)

col = ['SC_Nature', 'SC_Human', 'SC_Household', 'SC_Installation', 'SC_Signals', 'SC_Traffic', 'SC_Speech', 'SC_Music']
dataset[col] = dataset[col].apply(sc_range)

dataset.head()

In [ ]:
# ADd acoustic features from AcousticFeatures_SingleValues.csv dataset
path = '../dataset/AcousticFeatures_SingleValues.csv'
acoustic_dataset = pd.read_csv(path, sep=';')

# items to keep from acoustic dataset
columns_to_select = ['Key', 'Channel', 'LAeq_default', 'N5_default', 'FavgArith_default', 'RAavgArith', 'SavgArith_default', 'R_default', 'T_default']
acoustic_dataset = acoustic_dataset[columns_to_select]

acoustic_dataset_max_values = acoustic_dataset.groupby('Key').max().reset_index()
acoustic_dataset_max_values = acoustic_dataset_max_values.drop(columns=['Channel'])
acoustic_dataset_max_values.head()

# calculate mean of both audio channels
#acoustic_dataset = acoustic_dataset.groupby('Key').mean().reset_index()
# acoustic_dataset = acoustic_dataset.drop(columns=['Channel'])
# acoustic_dataset.head()

In [ ]:
# merge dataset and acoustic_dataset
final_dataset = dataset.merge(acoustic_dataset_max_values, left_on='file_name', right_on='Key', how='left')
final_dataset = final_dataset.drop(columns=['Key'])
final_dataset.head()

In [ ]:
# get min max ranges of acoustic features
# --> used for Sliders in WebApp

col = ['LAeq_default', 'N5_default', 'FavgArith_default', 'RAavgArith', 'SavgArith_default', 'R_default', 'T_default']

min_max_values = final_dataset[col].agg({'min', 'max'})
min_max_values

In [ ]:
# store dataset
final_dataset.to_csv('../dataset/FinalDataset.csv', sep=';', index=False)

#### Add time-variant acoustic features

In [ ]:
from mosqito.sq_metrics import sharpness_din_tv, loudness_zwtv, roughness_dw, pr_ecma_perseg, tnr_ecma_perseg
import numpy as np
import soundfile as sf

##### Compute calibration factor

In [ ]:
P_REF = 20e-6 

def sound_pressure(audio_data):
    p_rms = np.sqrt(np.mean(audio_data**2))  # RMS berechnen
    laeq = 20 * np.log10(p_rms / P_REF)  # Umrechnung in dB SPL
    return laeq

def calibration_value(audio_data, expected_l_ch1, expected_l_ch2):
    raw_laeg_ch1 = sound_pressure(audio_data[:, 0])
    raw_laeg_ch2 = sound_pressure(audio_data[:, 1])

    cal_ch1 = expected_l_ch1 - raw_laeg_ch1
    cal_ch2 = expected_l_ch2 - raw_laeg_ch2

    return cal_ch1, cal_ch2

def apply_calibration(audio_data, cal_ch1, cal_ch2):
    audio_calibrated = np.zeros_like(audio_data)
    audio_calibrated[:, 0] = audio_data[:, 0] * (10**(cal_ch1 / 20))  # dB-Wert in linear gain factor
    audio_calibrated[:, 1] = audio_data[:, 1] * (10**(cal_ch2 / 20)) 
    return audio_calibrated

# load audio
file_path = "../dataset/audiofiles/1132730_16.wav"
audio_data, fs = sf.read(file_path)

# callibrate
cal_ch1, cal_ch2 = calibration_value(audio_data, 64.13, 64.57)

##### Compute acoustic features

In [ ]:
# compute audio features and save to dataset
import os
import soundfile as sf

def compute_acoustic_features(audio, sr):

    N_time, _, _, _ = loudness_zwtv(audio, sr) # N_time
    S, _ = sharpness_din_tv(audio, fs=sr) # S
    R_time, _, _, _ = roughness_dw(audio, sr) # R_time
    t_pr, _, _, _, _ = pr_ecma_perseg(audio, sr) # t_pr
    t_tnr, _, _, _, _ = tnr_ecma_perseg(audio, sr) # t_tnr
        
    return N_time, S, R_time, t_pr, t_tnr

def scale_acoustic_features_to_audio_length(feature_array, audio_length):


    original_length = len(feature_array)
    block_size = original_length / audio_length
    result = np.zeros(audio_length)
    
    for i in range(audio_length):
        start_idx = int(i * block_size)
        end_idx = int(min((i + 1) * block_size, original_length))
        
        # Average the values in this block
        if start_idx < end_idx:
            result[i] = np.mean(feature_array[start_idx:end_idx])
        else:
            result[i] = feature_array[start_idx] if start_idx < original_length else 0
            
    return result

folder_path = '/dataset/audiofiles'
audio_files = [f for f in os.listdir(folder_path) if f.endswith(('.wav'))]
    
    # Process each audio file
for audio_file in audio_files:
    file_path = os.path.join(folder_path, audio_file)
    
    try:
        # Load audio
        audio_data, sr = sf.read(file_path)

        # calibrate audio and convert to mono
        calibrated_audio = apply_calibration(audio_data, cal_ch1, cal_ch2)
        mono_audio = np.mean(calibrated_audio, axis=1)

        # compute acoustic features
        loudness, sharpness, roughness, prominence_ratio, tone_to_noise_ratio = compute_acoustic_features(mono_audio, sr)

        # averagte features to audio length
        time_scaling_factor = 1
        length_audio = np.floor(len(mono_audio)/sr) + 1
        loudness = scale_acoustic_features_to_audio_length(loudness, int((length_audio*time_scaling_factor)))
        sharpness = scale_acoustic_features_to_audio_length(sharpness, int((length_audio*time_scaling_factor)))
        roughness = scale_acoustic_features_to_audio_length(roughness, int((length_audio*time_scaling_factor)))

        audio_features = {
            "loudness": loudness.tolist(),
            "sharpness": sharpness.tolist(),
            "roughness": roughness.tolist(),
            "prominence_ratio": prominence_ratio.tolist(),
            "tone_to_noise_ratio": tone_to_noise_ratio.tolist()
        }

        # push to dataset
        file_name = file_path.split('/')[-1].split('.')[0]
        row_idx = df[df['file_name'] == file_name].index.to_list()

        if row_idx:

            # store to your dataset
            df.at[row_idx[0], 'temporal_audio_features'] = [audio_features]
            print('\n done processing file ', audio_file)
            print('\n')
        else:
            print(f"No row found with file_name: {file_name}")
    except Exception as e:
        print(f"Error processing {audio_file}: {str(e)}")
